# Capstone Project - The Battle of Neighborhoods - Week 2
## Optimal Location for a New African Restaurant in New York City
Author: Eniola Aliu
## Table of contents

## <a id='another_cell'>1. Introduction: Business Problem</a> 

Anyone that works in the restaurant, service, and hospitality industry knows that it’s not a “get rich quick” type of career. It takes time, hard work, and dedication to be successful.  Even still, a study by Ohio State University reveals that nearly 60% of restaurants close or change ownership within the first year and 80% fail within the first 5 years.

Since there are lots of restaurants in New York City, we will try to detect **locations that are not already crowded with restaurants.** We are also particularly interested in areas with no African restaurants in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.


## Table of Contents  

[1. Introduction: Business Problem](#another_cell)   
[2. Data](#another_cell2)   
[3. Methodology](#another_cell3)  
[4. Analysis](#another_cell4)  
[5. Results](#another_cell5)     
[6. Discussion](#another_cell6)   
[7. Conclusion](#another_cell7)    

**Target audience:** This project aims to provide a solution for stakeholders who desire to find the optimal location to open a new african restaurant in Ney York City. 

## <a id='another_cell2'>2. Data</a> 

In order to solve this problem, the following will be required:

* New York City data containing the neighborhoods and boroughs, latitudes, and longitudes will be obtained from the data source: https://cocl.us/new_york_dataset
* All data related to locations and quality of Italian restaurants will be obtained via the FourSquare API utilized via the Request library in Python.
* Use the Geopy and Folium library to get the coordinates of every locations and map geospatial data on a London map.
* Using Foursquare API, collect the top 100 restaurants and their categories for each location within a radius 500 meters.
* Group collected restaurants by location and by taking the mean of the frequency of occurrence of each type, preparing them for clustering.
* Cluster restaurants by k-means algorithm and analyze the top 10 most common restaurants in each cluster.
* Visualize clusters on the map, thus showing the best locations for opening the chosen restaurant.

### [3. Methodology](#another_cell3)
* Data will be collected from https://cocl.us/new_york_dataset and cleaned and processed into a pandas dataframe.

* FourSquare API will be used to locate all venues and then filtered by African restaurants. Ratings, tips, and likes by users will be counted and added to the dataframe.

* Data will be sorted based on rankings.

* Finally, the data be will be visually assessed graphically from Python libraries.

In [1]:
# Importing all the necessary libraries we will be needing to do the Ananlysis
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import time # for time delay while working with API
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json # library to handle JSON files

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
# regular expressions
import re

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 122kB 6.2MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.3MB/s eta 0:00:01     |█████████████████▋              | 3.0MB 5.3MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##########

### Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

We can run a wget command and access the data. So let's go ahead and do that.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


### Load and explore the data
Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [5]:
relevant_data = newyork_data['features']
relevant_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Tranform the data into a pandas dataframe
The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [8]:
for data in relevant_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Let's examine the resulting dataframe.

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Let's quickly create a map of New York with neighborhoods superimposed on top.

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'GNAXGLCD2HMU1QX5CL04JNPAEM2YEJMKTRZCHFDCZICK2OCP' # your Foursquare ID
CLIENT_SECRET = 'C1HCZYM0INCRXSR5KBPTQGASPD24NDJGDAM3BL2XTGIZCR3O' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GNAXGLCD2HMU1QX5CL04JNPAEM2YEJMKTRZCHFDCZICK2OCP
CLIENT_SECRET:C1HCZYM0INCRXSR5KBPTQGASPD24NDJGDAM3BL2XTGIZCR3O


#### Now, let's get the top 100 venues that are in New York City within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [17]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GNAXGLCD2HMU1QX5CL04JNPAEM2YEJMKTRZCHFDCZICK2OCP&client_secret=C1HCZYM0INCRXSR5KBPTQGASPD24NDJGDAM3BL2XTGIZCR3O&v=20180604&ll=40.7127281,-74.0060152&radius=500&limit=100'

Send the GET request and examine the results

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1097f7d732ec1c90803029'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 106,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57f0689d498e7d49d9189369',
       'name': 'The Bar Room at Temple Court',
       'location': {'address': '123 Nassau St',
        'lat': 40.7114477287544,
        'lng': -74.00680157032005,
        'labe

Now we know that all the information is in the items key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Bar Room at Temple Court,Hotel Bar,40.711448,-74.006802
1,Alba Dry Cleaner & Tailor,Laundry Service,40.711434,-74.006272
2,"The Beekman, A Thompson Hotel",Hotel,40.711173,-74.006702
3,Gibney Dance Center Downtown,Dance Studio,40.713923,-74.005661
4,City Hall Park,Park,40.712415,-74.006724


And how many venues were returned by Foursquare?

In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Let's create a function to repeat the same process to all the neighborhoods in Newyork

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's write the code to run the above function on each neighborhood and create a new dataframe called newyork_venues.

In [24]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

Let's check the size of the resulting dataframe

In [25]:
print(newyork_venues.shape)
newyork_venues.head()

(10151, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's check how many restaurants were returned for each neighborhood.

In [26]:
newyork_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()

,Venue
Neighborhood,
Allerton,34
Annadale,13
Arden Heights,4
Arlington,8
Arrochar,21
Arverne,20
Astoria,100
Astoria Heights,12
Auburndale,19


And check if Foursquare API did not return restaurants for some locations.

In [27]:
x = newyork_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count().shape[0]
y = neighborhoods.shape[0]
empty_locations = []
if x != y:
    print('Missing data for {0} locations:'.format(y-x))
    # And print them
    for i in range(neighborhoods.shape[0]):
        loc = neighborhoods.iloc[i,0]
        k = 0
        for j in range(newyork_venues.shape[0]):
            if loc == newyork_venues.iloc[j,0]:
                k += 1
        if k == 0:
            print(i,loc)
            empty_locations.append(loc)

Missing data for 4 locations:
0 Bronx
1 Bronx
2 Bronx
3 Bronx
4 Bronx
5 Bronx
6 Manhattan
7 Bronx
8 Bronx
9 Bronx
10 Bronx
11 Bronx
12 Bronx
13 Bronx
14 Bronx
15 Bronx
16 Bronx
17 Bronx
18 Bronx
19 Bronx
20 Bronx
21 Bronx
22 Bronx
23 Bronx
24 Bronx
25 Bronx
26 Bronx
27 Bronx
28 Bronx
29 Bronx
30 Bronx
31 Bronx
32 Bronx
33 Bronx
34 Bronx
35 Bronx
36 Bronx
37 Bronx
38 Bronx
39 Bronx
40 Bronx
41 Bronx
42 Bronx
43 Bronx
44 Bronx
45 Bronx
46 Brooklyn
47 Brooklyn
48 Brooklyn
49 Brooklyn
50 Brooklyn
51 Brooklyn
52 Brooklyn
53 Brooklyn
54 Brooklyn
55 Brooklyn
56 Brooklyn
57 Brooklyn
58 Brooklyn
59 Brooklyn
60 Brooklyn
61 Brooklyn
62 Brooklyn
63 Brooklyn
64 Brooklyn
65 Brooklyn
66 Brooklyn
67 Brooklyn
68 Brooklyn
69 Brooklyn
70 Brooklyn
71 Brooklyn
72 Brooklyn
73 Brooklyn
74 Brooklyn
75 Brooklyn
76 Brooklyn
77 Brooklyn
78 Brooklyn
79 Brooklyn
80 Brooklyn
81 Brooklyn
82 Brooklyn
83 Brooklyn
84 Brooklyn
85 Brooklyn
86 Brooklyn
87 Brooklyn
88 Brooklyn
89 Brooklyn
90 Brooklyn
91 Brooklyn
92 Brookly

Let's find out how many unique categories can be curated from all the returned restaurants.

In [28]:
print('There are {0} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 426 uniques categories.


## 4. Analysis 
Let's perform some basic explanatory data analysis and derive some additional info from our raw data.

### Exploring Restaurants


To begin analysis we need to transform collected information using the one-hot encoding method.

In [29]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add location column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move location column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle Hous

And let's examine the new dataframe size.

In [30]:
newyork_onehot.shape

(10151, 426)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category preparing the dataframe for clustering.

In [31]:
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle Hous

Let's confirm the new size.

In [32]:
newyork_grouped.shape

(302, 426)

Let's investigate each neighborhood along with the top 5 most common venues.

In [33]:
# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Deli / Bodega,Department Store,Pharmacy,Chinese Restaurant,Bakery,Check Cashing Service,Grocery Store,Martial Arts Dojo
1,Annadale,Pizza Place,Bar,Restaurant,Pub,Train Station,Pharmacy,Food,Cosmetics Shop,American Restaurant,Liquor Store
2,Arden Heights,Bus Stop,Coffee Shop,Pharmacy,Pizza Place,Women's Store,Fast Food Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
3,Arlington,Bus Stop,Intersection,Home Service,Deli / Bodega,Boat or Ferry,American Restaurant,Grocery Store,Falafel Restaurant,Fast Food Restaurant,Farmers Market
4,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Athletics & Sports,Nail Salon,Supermarket,Middle Eastern Restaurant,Mediterranean Restaurant,Outdoors & Recreation,Bagel Shop
5,Arverne,Surf Spot,Sandwich Place,Metro Station,Restaurant,Café,Bed & Breakfast,Coffee Shop,Board Shop,Beach,Donut Shop
6,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Mediterranean Restaurant,Hookah Bar,Seafood Restaurant,Pizza Place,Bakery,Indian Restaurant,Café
7,Astoria Heights,Burger Joint,Playground,Bakery,Bowling Alley,Supermarket,Hostel,Pizza Place,Business Service,Plaza,Italian Restaurant
8,Auburndale,Italian Restaurant,Miscellaneous Shop,Bar,Toy / Game Store,Pet Store,Pharmacy,Korean Restaurant,Supermarket,Fast Food Restaurant,Athletics & Sports
9,Bath Beach,Pizza Place,Pharmacy,Italian Restaurant,Deli / Bodega,Bubble Tea Shop,Fast Food Restaurant,Cantonese Restaurant,Chinese Restaurant,Gas Station,Park


### Clustering Restaurants
Run k-means to cluster the neighborhood into 5 clusters.

In [35]:
# set number of clusters
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 0, 0, 0, 0, 0, 4, 0, 4], dtype=int32)

Let's create a new dataframe that includes the clusters as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

newyork_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

newyork_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,Pharmacy,Laundromat,Sandwich Place,Donut Shop,Deli / Bodega,Ice Cream Shop,Gas Station,Dessert Shop,Farmers Market,Farm
1,Bronx,Co-op City,40.874294,-73.829939,4,Bus Station,Post Office,Discount Store,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Park,Bagel Shop,Pharmacy,Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,4,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Juice Bar,Chinese Restaurant,Metro Station,Automotive Shop,Fast Food Restaurant,Seafood Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,4,Cosmetics Shop,Bus Station,Plaza,River,Women's Store,Field,Event Service,Event Space,Exhibit,Eye Doctor
4,Bronx,Riverdale,40.890834,-73.912585,4,Park,Bus Station,Food Truck,Plaza,Bank,Gym,Baseball Field,Flower Shop,Food,Food Stand


### 5. Results
And now we are ready to conclude our report.
#### Examine Clusters
Let's examine each cluster and the discriminating restaurant categories that distinguish a cluster.

**Cluster 1**

In [37]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Marble Hill,Gym,Coffee Shop,Discount Store,Steakhouse,Miscellaneous Shop,Supplement Shop,Shopping Mall,Big Box Store,Seafood Restaurant,Tennis Stadium
7,Woodlawn,Pub,Food & Drink Shop,Pizza Place,Bar,Deli / Bodega,Playground,Bakery,Rental Car Location,Cosmetics Shop,Plaza
9,Williamsbridge,Bar,Nightclub,Caribbean Restaurant,Soup Place,Fast Food Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop
12,City Island,Thrift / Vintage Store,Park,Seafood Restaurant,Grocery Store,French Restaurant,Spanish Restaurant,Baseball Field,Harbor / Marina,Boat or Ferry,Café
22,Port Morris,Furniture / Home Store,Latin American Restaurant,Distillery,Metro Station,Donut Shop,Grocery Store,Peruvian Restaurant,Restaurant,Spanish Restaurant,Food Truck
24,Hunts Point,Spanish Restaurant,Café,Seafood Restaurant,Grocery Store,Bank,Gourmet Shop,BBQ Joint,Farmers Market,Juice Bar,Restaurant
27,Clason Point,Park,Grocery Store,Boat or Ferry,South American Restaurant,Bus Stop,Convenience Store,Pool,Exhibit,Fast Food Restaurant,Eye Doctor
28,Throgs Neck,Coffee Shop,Liquor Store,American Restaurant,Sports Bar,Bar,Pizza Place,Asian Restaurant,Deli / Bodega,Italian Restaurant,Food Stand
34,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Sandwich Place,Grocery Store,Dessert Shop,Bank,Donut Shop,Shoe Store
39,Edgewater Park,Italian Restaurant,Pizza Place,Deli / Bodega,Spa,Park,Liquor Store,Coffee Shop,Bar,Donut Shop,Asian Restaurant


In [38]:
cluster_1 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]
cluster_1.describe(include='all')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,157,157,157,157,157,157,157,157,157,157,157
unique,153,53,69,75,76,74,78,76,89,81,91
top,Chelsea,Italian Restaurant,Coffee Shop,Pizza Place,Pizza Place,Bakery,Pizza Place,Bakery,Exhibit,Exhibit,Eye Doctor
freq,2,26,12,12,12,12,8,9,8,8,8


**Cluster 2**

In [39]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Women's Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm
203,Todt Hill,Park,Women's Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm


In [40]:
cluster_2 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]
cluster_2.describe(include='all')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,2,2,2,2,2,2,2,2,2,2,2
unique,2,1,1,1,1,1,1,1,1,1,1
top,Todt Hill,Park,Women's Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm
freq,1,2,2,2,2,2,2,2,2,2,2


**Cluster 3**

In [41]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Sea Gate,Home Service,Beach,Spa,Bus Station,American Restaurant,Women's Store,Exhibit,Eye Doctor,Fabric Shop,Factory
172,Breezy Point,Beach,Supermarket,Bus Stop,Monument / Landmark,Trail,Women's Store,Field,Exhibit,Eye Doctor,Fabric Shop
178,Rockaway Beach,Beach,Ice Cream Shop,Deli / Bodega,BBQ Joint,Bagel Shop,Latin American Restaurant,Seafood Restaurant,Arepa Restaurant,Metro Station,Bar
179,Neponsit,Beach,Bar,Field,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
190,Belle Harbor,Beach,Pub,Deli / Bodega,Spa,Bakery,Pharmacy,Donut Shop,Boutique,Bagel Shop,Mexican Restaurant
193,Brookville,Deli / Bodega,Women's Store,Filipino Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
204,South Beach,Pier,Beach,Deli / Bodega,Athletics & Sports,Filipino Restaurant,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
232,Midland Beach,Beach,Deli / Bodega,Bookstore,Basketball Court,Bus Stop,Baseball Field,Chinese Restaurant,Liquor Store,Pet Store,Home Service
288,Roxbury,Irish Pub,Deli / Bodega,Beach,Trail,Fast Food Restaurant,Baseball Field,Field,Event Space,Exhibit,Eye Doctor
302,Hammels,Beach,Food Truck,Gym / Fitness Center,Dog Run,Bus Stop,Bus Station,Diner,Shoe Store,Deli / Bodega,Fried Chicken Joint


In [42]:
cluster_3 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]
cluster_3.describe(include='all')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,10,10,10,10,10,10,10,10,10,10,10
unique,10,5,8,8,9,8,6,7,8,8,8
top,Breezy Point,Beach,Beach,Deli / Bodega,Event Service,Bus Stop,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant
freq,1,6,2,3,2,2,3,3,3,3,3


**Cluster 4**

In [43]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
207,Port Ivory,Bar,Women's Store,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm


In [44]:
cluster_4 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]
cluster_4.describe(include='all')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,1,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1,1,1,1
top,Port Ivory,Bar,Women's Store,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm
freq,1,1,1,1,1,1,1,1,1,1,1


**Cluster 5**

In [45]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Laundromat,Sandwich Place,Donut Shop,Deli / Bodega,Ice Cream Shop,Gas Station,Dessert Shop,Farmers Market,Farm
1,Co-op City,Bus Station,Post Office,Discount Store,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Park,Bagel Shop,Pharmacy,Restaurant
2,Eastchester,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Juice Bar,Chinese Restaurant,Metro Station,Automotive Shop,Fast Food Restaurant,Seafood Restaurant
3,Fieldston,Cosmetics Shop,Bus Station,Plaza,River,Women's Store,Field,Event Service,Event Space,Exhibit,Eye Doctor
4,Riverdale,Park,Bus Station,Food Truck,Plaza,Bank,Gym,Baseball Field,Flower Shop,Food,Food Stand
5,Kingsbridge,Pizza Place,Bar,Latin American Restaurant,Mexican Restaurant,Supermarket,Donut Shop,Spanish Restaurant,Sandwich Place,Pharmacy,Fried Chicken Joint
8,Norwood,Pizza Place,Park,Bank,Pharmacy,Chinese Restaurant,Deli / Bodega,Mobile Phone Shop,Fast Food Restaurant,Restaurant,Caribbean Restaurant
10,Baychester,Donut Shop,Fried Chicken Joint,Mattress Store,Shopping Mall,Spanish Restaurant,Sandwich Place,Men's Store,Electronics Store,Bank,Mexican Restaurant
11,Pelham Parkway,Bus Station,Italian Restaurant,Pizza Place,Metro Station,Bank,Coffee Shop,Liquor Store,Gym / Fitness Center,Chinese Restaurant,Gift Shop
13,Bedford Park,Deli / Bodega,Diner,Chinese Restaurant,Pizza Place,Mexican Restaurant,Sandwich Place,Pub,Smoke Shop,Gym,Grocery Store


In [46]:
cluster_5 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]
cluster_5.describe(include='all')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,136,136,136,136,136,136,136,136,136,136,136
unique,136,43,52,59,68,66,73,67,75,73,72
top,Crown Heights,Pizza Place,Chinese Restaurant,Pizza Place,Pizza Place,Sandwich Place,Sandwich Place,Pizza Place,Fast Food Restaurant,Bank,Eye Doctor
freq,1,22,11,10,13,11,7,7,6,9,7


### Visualizing Clusters
Finally, let's visualize the resulting clusters.

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**MAP LEGEND**  
Cluster 1 - red dots  
Cluster 2 - purple dots  
Cluster 3 - blue dots  
Cluster 4 - green dots  
Cluster 5 - orange dots

## 6. Discussion
Analyzing the most popular restaurants in each cluster, the stakeholder should prefer the least popular types as a safe choice. There are few African Restaurants in New York City which makes it a **very lucrative** business. There is no sense in opening an African Restaurant at Todt Hill in the same street where there is another Ethopian Restaurant. Of course, there might be more than 10 types in a location. And one might object, that following this logic, the stakeholder must prefer the last type in a full list, and not the 10th one. But bear in mind that descending on the popularity list we might face an absence of demand for this type of food, and open a restaurant that is not needed in this particular location. Presence of interested customers is a must for a successful business. That is why in our recommendations we offer to stop on 10th and 9th positions.

Recommendations, based on description of each cluster:  
**Cluster 1 Locations:** Seafood Restaurant  
**Cluster 2 Locations:** Falafel Restaurant or Ethopian Restaurant  
**Cluster 3 Locations:** Falafel Restaurant  
**Cluster 4 Locations:** Falafel Restaurant  
**Cluster 5 Locations:** Fast Food Restaurant or Chinese Restaurant  

After the type of restaurant is chosen, it is time to select a right place. Using the map created in 5.2 and its legend the solution is quite obvious.

## 7. Conclusion
In this report we worked out a methodology to determine what the most promising type of restaurant is and where it should be opened.

We collected information about New york city boroughs from Wikipedia, and using **geospatial libraries** mapped them. Using **Foursquare API**, we collected the top 100 restaurants and their types for each location within a radius 500 meters from its central point. Then we grouped collected restaurants by location and by taking the mean of the frequency of occurrence of each type, preparing them for clustering. Finally we clustered restaurants by the **k-means algorithm** and analyze the top 10 most common restaurants in each cluster, making useful observations. Eventually we visualized clusters on the map, thus showing the best locations for opening the chosen type of restaurant.  

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.

This type of analysis can be applied to any city of your choice that has available geospatial information.

This type of analysis can be applied to any type of venue (shopping, clubs, etc.) that is available in Foursquare database.